# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/bitcoin/1_backfill_feature_groups.ipynb)

## 🗒️ This notebook is divided into the following sections:
1. Fetch historical data 
2. Connect to the Hopsworks feature store
3. Create feature groups and insert them to the feature store

![tutorial-flow](../../images/01_featuregroups.png)

---
## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

### Don't forget to create an `.env` configuration file where all the necessary environment variables (API keys) will be stored:
![](images/api_keys_env_file.png)

---

---

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [ ]:
!pip install -U unicorn-binance-rest-api --quiet
!pip install -U python-dotenv --quiet

In [ ]:
import pandas as pd

from functions import *

___

## <span style="color:#ff5f27;"> 💽 Loading Data</span>

### <span style='color:#ff5f27'> 📈 Bitcoin Data

In [ ]:
df_bitcoin = parse_btc_data(number_of_days_ago=2000)

df_bitcoin = df_bitcoin[(df_bitcoin.date >= '2021-02-05 10:00:00') & (df_bitcoin.date <= '2022-06-04 23:00:00')] 
df_bitcoin.reset_index(drop=True,inplace=True)

df_bitcoin.head(3)

In [ ]:
df_bitcoin_processed = process_btc_data(df_bitcoin)
df_bitcoin_processed.tail(3)

> Older records may come with time=11pm or time=9pm, but new ones have time=10pm. Thats because of timezones and daylight saving time. Lets apply this function to make unix column usable.

In [ ]:
def get_hours(unix):
    return unix / 3600000 % 24

In [ ]:
def fix_unix(unix):
    if get_hours(unix) == 23.0:
        return unix - 3600000
    elif get_hours(unix) == 21.0:
        return unix + 3600000
    return unix

In [ ]:
get_hours(1654293600000)

In [ ]:
df_bitcoin_processed.unix = df_bitcoin_processed.unix.apply(fix_unix)

### <span style='color:#ff5f27'> 💭 Tweets Data

In [ ]:
tweets_textblob = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_textblob.csv", index_col=0)
tweets_textblob.unix = tweets_textblob.unix.apply(fix_unix)
tweets_textblob.head(3)

In [ ]:
tweets_vader = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_vader.csv", index_col=0)
tweets_vader.unix = tweets_vader.unix.apply(fix_unix)
tweets_vader.tail(3)

---

## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

### <span style='color:#ff5f27'> 📈 Bitcoin Price Feature Group

In [ ]:
btc_price_fg = fs.get_or_create_feature_group(
    name='bitcoin_price',
    description='Bitcoin price aggregated for days',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix'
)

btc_price_fg.insert(df_bitcoin_processed)

### <span style='color:#ff5f27'> 💭 Tweets Feature Groups

In [ ]:
tweets_textblob_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_textblob',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix'
)

tweets_textblob_fg.insert(tweets_textblob)

In [ ]:
tweets_vader_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_vader',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time='unix'
)

tweets_vader_fg.insert(tweets_vader)

---